# Родионов   Д.А.
ИУ5-65Б Вариант №12

Импортируем библиотеки:

In [13]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_blobs, make_circles
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

In [26]:
from google.colab import drive
drive.mount("/content/gdrive")
data = pd.read_csv('/content/gdrive/My Drive/occupancy_data/dc-wikia-data.csv', sep=",")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
data

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891,66302,Nadine West (New Earth),\/wiki\/Nadine_West_(New_Earth),Public Identity,Good Characters,NaN,NaN,Female Characters,NaN,Living Characters,NaN,NaN,NaN
6892,283475,Warren Harding (New Earth),\/wiki\/Warren_Harding_(New_Earth),Public Identity,Good Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN
6893,283478,William Harrison (New Earth),\/wiki\/William_Harrison_(New_Earth),Public Identity,Good Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN
6894,283471,William McKinley (New Earth),\/wiki\/William_McKinley_(New_Earth),Public Identity,Good Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN


# Обработка пропусков

In [28]:
data.keys().to_list()

['page_id',
 'name',
 'urlslug',
 'ID',
 'ALIGN',
 'EYE',
 'HAIR',
 'SEX',
 'GSM',
 'ALIVE',
 'APPEARANCES',
 'FIRST APPEARANCE',
 'YEAR']

In [29]:
data.isnull().sum()

page_id                0
name                   0
urlslug                0
ID                  2013
ALIGN                601
EYE                 3628
HAIR                2274
SEX                  125
GSM                 6832
ALIVE                  3
APPEARANCES          355
FIRST APPEARANCE      69
YEAR                  69
dtype: int64

In [30]:
data.shape

(6896, 13)

In [31]:
total_count = data.shape[0]
print('Всего строк: {}'.format(total_count))

Всего строк: 6896


In [32]:
data = data.dropna(axis=0, how='any')
data.shape

(38, 13)

In [33]:
data.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
48,1704,John Constantine (New Earth),\/wiki\/John_Constantine_(New_Earth),Public Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Bisexual Characters,Living Characters,371.0,"1984, June",1984.0
65,8856,Renee Montoya (New Earth),\/wiki\/Renee_Montoya_(New_Earth),Secret Identity,Good Characters,Brown Eyes,Black Hair,Female Characters,Homosexual Characters,Living Characters,308.0,"1992, March",1992.0
119,1862,Todd Rice (New Earth),\/wiki\/Todd_Rice_(New_Earth),Public Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,Homosexual Characters,Living Characters,208.0,"1983, September",1983.0
173,1597,Hartley Rathaway (New Earth),\/wiki\/Hartley_Rathaway_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Red Hair,Male Characters,Homosexual Characters,Living Characters,160.0,"1959, May",1959.0
253,10139,Holly Robinson (New Earth),\/wiki\/Holly_Robinson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Red Hair,Female Characters,Homosexual Characters,Living Characters,114.0,"1987, February",1987.0


# Кодирование категориальных признаков

Удалим колонки, которые не влияют на целевой признак:

In [35]:
data = data.drop(columns='name')
data = data.drop(columns='urlslug')
data = data.drop(columns='FIRST APPEARANCE')

In [36]:
data.shape

(38, 10)

In [37]:
data.head()

,page_id,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,YEAR
48,1704,Public Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Bisexual Characters,Living Characters,371.0,1984.0
65,8856,Secret Identity,Good Characters,Brown Eyes,Black Hair,Female Characters,Homosexual Characters,Living Characters,308.0,1992.0
119,1862,Public Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,Homosexual Characters,Living Characters,208.0,1983.0
173,1597,Secret Identity,Good Characters,Blue Eyes,Red Hair,Male Characters,Homosexual Characters,Living Characters,160.0,1959.0
253,10139,Secret Identity,Good Characters,Blue Eyes,Red Hair,Female Characters,Homosexual Characters,Living Characters,114.0,1987.0


In [38]:
data.dtypes

page_id          int64
ID              object
ALIGN           object
EYE             object
HAIR            object
SEX             object
GSM             object
ALIVE           object
APPEARANCES    float64
YEAR           float64
dtype: object

In [39]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le = LabelEncoder()
df_int = le.fit_transform(data['ID'])
data['ID'] = df_int
df_int = le.fit_transform(data['ALIGN'])
data['ALIGN'] = df_int
df_int = le.fit_transform(data['EYE'])
data['EYE'] = df_int
df_int = le.fit_transform(data['HAIR'])
data['HAIR'] = df_int
df_int = le.fit_transform(data['SEX'])
data['SEX'] = df_int
df_int = le.fit_transform(data['GSM'])
data['GSM'] = df_int
df_int = le.fit_transform(data['ALIVE'])
data['ALIVE'] = df_int
data.head()

,page_id,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,YEAR
48,1704,0,1,1,1,1,0,1,371.0,1984.0
65,8856,1,1,2,0,0,1,1,308.0,1992.0
119,1862,0,1,2,3,1,1,1,208.0,1983.0
173,1597,1,1,1,5,1,1,1,160.0,1959.0
253,10139,1,1,1,5,0,1,1,114.0,1987.0


In [40]:
sc1 = MinMaxScaler()
data['ID'] = sc1.fit_transform(data[['ID']])
data['ALIGN'] = sc1.fit_transform(data[['ALIGN']])
data['EYE'] = sc1.fit_transform(data[['EYE']])
data['HAIR'] = sc1.fit_transform(data[['HAIR']])
data['SEX'] = sc1.fit_transform(data[['SEX']])
data['GSM'] = sc1.fit_transform(data[['GSM']])
data['ALIVE'] = sc1.fit_transform(data[['ALIVE']])
data.head()

,page_id,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,YEAR
48,1704,0.0,0.5,0.25,0.166667,1.0,0.0,1.0,371.0,1984.0
65,8856,1.0,0.5,0.50,0.000000,0.0,1.0,1.0,308.0,1992.0
119,1862,0.0,0.5,0.50,0.500000,1.0,1.0,1.0,208.0,1983.0
173,1597,1.0,0.5,0.25,0.833333,1.0,1.0,1.0,160.0,1959.0
253,10139,1.0,0.5,0.25,0.833333,0.0,1.0,1.0,114.0,1987.0


# Разделение на обучающую и тестовую выборки

In [52]:
target = data['ALIVE']
data_X_train, data_X_test, data_y_train, data_y_test = train_test_split(
    data, target, test_size=0.2, random_state=1)

In [53]:
data_X_train.shape, data_y_train.shape

((30, 10), (30,))

In [54]:
data_X_test.shape, data_y_test.shape

((8, 10), (8,))

In [55]:
np.unique(target)

array([0., 1.])

# Метод опорных векторов

In [56]:
svr_1 = LinearSVC()
svr_1.fit(data_X_train, data_y_train)

Liblinear failed to converge, increase the number of iterations.


LinearSVC()

In [57]:
data_y_pred_1 = svr_1.predict(data_X_test)

In [58]:
accuracy_score(data_y_test, data_y_pred_1)

0.875

In [59]:
f1_score(data_y_test, data_y_pred_1, average='micro')

0.875

In [60]:
f1_score(data_y_test, data_y_pred_1, average='macro')

0.4666666666666667

In [61]:
f1_score(data_y_test, data_y_pred_1, average='weighted')

0.8166666666666667

In [62]:
svr_2 = LinearSVC(C=1.0, max_iter=10000)
svr_2.fit(data_X_train, data_y_train)

Liblinear failed to converge, increase the number of iterations.


LinearSVC(max_iter=10000)

In [63]:
data_y_pred_2 = svr_2.predict(data_X_test)

In [64]:
accuracy_score(data_y_test, data_y_pred_2)

0.875

In [65]:
f1_score(data_y_test, data_y_pred_2, average='micro')

0.875

In [66]:
f1_score(data_y_test, data_y_pred_2, average='macro')

0.4666666666666667

In [67]:
f1_score(data_y_test, data_y_pred_2, average='weighted')

0.8166666666666667

In [68]:
svr_3 = LinearSVC(C=1.0, penalty='l1', dual=False, max_iter=10000)
svr_3.fit(data_X_train, data_y_train)

LinearSVC(dual=False, max_iter=10000, penalty='l1')

In [69]:
data_y_pred_3_0 = svr_3.predict(data_X_train)
accuracy_score(data_y_train, data_y_pred_3_0)

1.0

In [70]:
data_y_pred_3 = svr_3.predict(data_X_test)
accuracy_score(data_y_test, data_y_pred_3)

1.0

In [71]:
f1_score(data_y_test, data_y_pred_3, average='micro')

1.0

In [72]:
f1_score(data_y_test, data_y_pred_3, average='macro')

1.0

In [73]:
f1_score(data_y_test, data_y_pred_3, average='weighted')

1.0

# Градиентный бустинг

In [74]:
ab1 = AdaBoostClassifier()
ab1.fit(data_X_train, data_y_train)

AdaBoostClassifier()

In [75]:
data_y_pred_1 = ab1.predict(data_X_test)

In [76]:
data_y_pred_1_0 = ab1.predict(data_X_train)

In [77]:
accuracy_score(data_y_train, data_y_pred_1_0)

1.0

In [78]:
accuracy_score(data_y_test, data_y_pred_1)

1.0

In [79]:
f1_score(data_y_test, data_y_pred_1, average='micro')

1.0

In [80]:
f1_score(data_y_test, data_y_pred_1, average='macro')

1.0

In [81]:
f1_score(data_y_test, data_y_pred_1, average='weighted')

1.0

Градиентный бустинг показал лучше качество, чем метод опорных векторов 